In [ ]:
!pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

# Cria uma variável de ambiente
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
for model in client.models.list():
  print(model.name)

models/chat-bison-001
models/text-bison-001
models/embedding-gecko-001
models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash-preview-04-17
models/gemini-2.5-flash-preview-04-17-thinking
models/gemini-2.5-pro-preview-05-06
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-preview-image-generation
models/gemini-2.0-flash-lite-p

In [ ]:
# Instalar Framework ADK de agentes do Google ################################################
%pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
#Integração com Google Tarefas e Google Agenda
from googleapiclient.discovery import build

def criar_tarefa_google_tarefas(titulo, notas=None, lista_id="default"):
    """
    Função INCOMPLETA para criar uma tarefa no Google Tarefas.
    A autenticação e a configuração correta do serviço são necessárias.
    """
    try:
        # SERVIÇO DO GOOGLE TAREFAS (requer autenticação)
        # service = build('tasks', 'v1', credentials=credentials)
        service = None # Substitua pela inicialização correta do serviço autenticado

        if service:
            task = {'title': titulo}
            if notas:
                task['notes'] = notas

            result = service.tasks().insert(tasklist=lista_id, body=task).execute()
            print(f"Tarefa criada: {result.get('title')}")
        else:
            print("Serviço do Google Tarefas não inicializado (autenticação pendente).")

    except Exception as e:
        print(f"Ocorreu um erro ao criar a tarefa: {e}")

def criar_evento_google_agenda(summary, start_time, end_time, description=None):
    """
    Função INCOMPLETA para criar um evento no Google Agenda.
    A autenticação e a configuração correta do serviço são necessárias.
    """
    try:
        # SERVIÇO DO GOOGLE AGENDA (requer autenticação)
        # service = build('calendar', 'v3', credentials=credentials)
        service = None # Substitua pela inicialização correta do serviço autenticado

        if service:
            event = {
                'summary': summary,
                'start': {
                    'dateTime': start_time,
                    'timeZone': 'America/Sao_Paulo', # Ajuste o fuso horário conforme necessário
                },
                'end': {
                    'dateTime': end_time,
                    'timeZone': 'America/Sao_Paulo', # Ajuste o fuso horário conforme necessário
                },
                'description': description,
                'reminders': {
                    'useDefault': False,
                    'overrides': [
                        {'method': 'popup', 'minutes': 30},
                    ],
                },
            }

            created_event = service.events().insert(calendarId='primary', body=event).execute()
            print(f"Evento criado: {created_event.get('htmlLink')}")
        else:
            print("Serviço do Google Agenda não inicializado (autenticação pendente).")

    except Exception as e:
        print(f"Ocorreu um erro ao criar o evento: {e}")

In [ ]:
##########################################
# --- Agente 1: Categoriza os inputs --- #
##########################################

def agente_categorizador(texto):
    categorizador = Agent(
        name="agente_categorizador",
        model="gemini-2.0-flash",
        description= "Agente que categoriza os inputs em  'Tarefa', 'Lembrete', 'Ideia', 'Projetos', 'Compromissos' ou 'Outro'",
        tools= [google_search],
        instruction="""
        Você é especialista em produtividade e organização. Utilizando técnicas de Processamento de Linguagem Natural (PLN), você deverá identificar
        palavras-chave, frases e o contexto de cada item para atribuir categorias relevantes. Algumas categorias iniciais poderiam ser:
        Tarefas/Ações: Coisas que precisam ser feitas.
        Lembretes: Informações importantes para recordar. Assim como data limite de entrega de trabalhos, por exemplo.
        Ideias/Projetos: Conceitos que podem ser desenvolvidos.
        Pensamentos Aleatórios: Reflexões, opiniões, etc. (que podem ser filtrados ou armazenados separadamente).
        Informações: Fatos, dados, etc.
        Compromissos: Eventos que possuem data e hora marcada.
        Lista de Compras: Itens que precisam ser comprados.
        """
    )
    entrada_do_agente_categorizador = f"Texto: {texto}"
    # Executa o agente
    itens_categorizados = call_agent(categorizador, entrada_do_agente_categorizador)
    return itens_categorizados


In [ ]:
#####################################################
# --- Agente 2: Organiza os itens categorizados --- #
#####################################################

def agente_organizador(itens_categorizados):
    organizador = Agent(
        name="agente_organizador",
        model="gemini-2.0-flash",
        description= "Agente que organiza os itens categorizados de forma estruturada",
        tools= [google_search],
        instruction="""
        Você irá identificar, para a categoria "Tarefas/Ações", verbos de ação e sugerir a criação de listas de tarefas com prazos (se mencionados) ou
        níveis de prioridade (com base em palavras como "urgente", "importante").
        Para "Ideias/Projetos", poderia ajudar a definir um escopo inicial ou identificar os primeiros passos.
        Para "Lembretes", poderia sugerir a criação de alertas ou anotações em um calendário.
        Para "Compromissos, poderia sugerir a criação de eventos no calendário.
        Possíveis Técnicas: Extração de informações, modelagem de tópicos, geração de listas e estruturas.
        """
    )

    entrada_do_organizador = f"Itens Categorizados: {itens_categorizados}"

    # Executa o agente
    itens_estruturados = call_agent(organizador, entrada_do_organizador)

    return itens_estruturados

In [ ]:
##############################################################
# --- Agente 3: Prioriza e Planeja os Itens Estruturados --- #
##############################################################

def agente_planejador(itens_estruturados):
    planejador = Agent(
        name="agente_planejador",
        model="gemini-2.0-flash",
        description= "Agente que prioriza e planeja os itens estruturados",
        tools= [google_search],
        instruction="""
        Você irá analisar a urgência, importância e o contexto das tarefas e projetos para sugerir uma ordem de prioridade.
        Você também irá ajudar a quebrar projetos maiores em tarefas menores e mais gerenciáveis.
        Possíveis Técnicas: Algoritmos de priorização (como a matriz de Eisenhower), planejamento de projetos básico.
        """
    )

    entrada_do_planejador = f"Itens Estruturados: {itens_estruturados}"

    # Executa o agente
    itens_planejados = call_agent(planejador, entrada_do_planejador)
    return itens_planejados

In [ ]:
###########################################
# --- Agente 4: Apresenta o resultado --- #
###########################################

def agente_apresentador(itens_planejados):
    apresentador = Agent(
        name="agente_final",
        model="gemini-2.0-flash",
        description= "Agente que apresenta o resultado!",
        tools= [google_search],
        instruction="""
        Você vai apresentar os resultados de forma organizada para que o usuário possa entender e agir. Você é especialista em produtividade e organização,
        assim, irá sugerir a forma mais prática e objetiva de como o usuário poderá agir. Entregue o resultado de forma organizada. Ao final, sugira onde cada
        item deverá ser alocado: lista de compras, tarefas e projetos (já com a lista de tarefas sugeridas nos itens planejados) - Google Tarefas; compromissos e lembretes - Google Agenda; Ideia e outros - Google Keep.
        """
    )

    entrada_do_apresentador = f"Itens Planejados: {itens_planejados}"

    # Executa o agente
    resultado_final = call_agent(apresentador, entrada_do_apresentador)
    return resultado_final


In [ ]:
print("🚀 Iniciando o Sistema de Libere a sua Mente! 🚀")

# --- Obter o Input do Usuário ---
texto = input("❓ Por favor, digite tudo o que estiver em sua mente! ")

# Inserir lógica do sistema de agentes ################################################
if not texto:
  print("Você esqueceu de digitar o texto!")
else:
  print(f"Maravilha! Vamos esvaziar essa mente e colocar tudo em ordem!")
  # Chama o primeiro agente - Categorizador
  dump_de_memoria= agente_categorizador (texto)
  #print("\n--- Resultado do agente 1 (Categorizador) ---\n")
  #display(to_markdown(dump_de_memoria))
  #print("-------------------------------------------------------------")

  # Chama o segundo agente - Organizador
  itens_organizados = agente_organizador(dump_de_memoria)
  #print("\n--- Resultado do agente 2 (Categorizador) ---\n")
  #display(to_markdown(itens_organizados))
  # print("-------------------------------------------------------------")

  # Chama o terceiro agente - Planejador
  planejamento = agente_planejador(itens_organizados)
  #print("\n--- Resultado do agente 3 (Planejador) ---\n")
  #display(to_markdown(planejamento))
  #print("-------------------------------------------------------------")

  # Chama o quarto agente - Apresentador
  resultado = agente_apresentador(planejamento)
  print("\n--- Resultado do agente 4 (Apresentador) ---\n")
  display(to_markdown(resultado))
  print("-------------------------------------------------------------")



🚀 Iniciando o Sistema de Libere a sua Mente! 🚀
❓ Por favor, digite tudo o que estiver em sua mente!  Estou pensando em fazer um intercâmbio, talvez para a Inglaterra. Na próxima semana tem um evento legal de astronomia, no dia 25 às 19h.   Preciso comprar banana, maçã e limão. Na segunda-feira tenho que ligar para o João.  Estou pensando em um novo projeto, talvez trabalhar com IA.      preciso voltar a ler, comprei alguns livros mas ainda não os comecei. Tambén preciso melhorar meus tempos para estudos, tanto para desenho como   para automação de testes. Treino ciclismo sempre pela manha em torno das 5h até 8h, menos as segundas-feiras. Preciso separar roupas para doação e comprar novas. Preciso consertar algumas coisas da casa como as tomadas e a cortina rasgada da sala. Preciso arrumar a planta da sala, colocar um tutor e criar um muda pois está muito grande.
Maravilha! Vamos esvaziar essa mente e colocar tudo em ordem!

--- Resultado do agente 4 (Apresentador) ---



> Com base na sua lista de itens planejados, aqui está uma sugestão de como alocar cada item nas ferramentas do Google para otimizar sua organização e produtividade:
> 
> **Google Tarefas:**
> 
> *   **Listas de Compras:**
>     *   Crie uma lista chamada "Mercado" e adicione: Banana, Maçã, Limão.
>     *   Crie uma lista chamada "Roupas" e adicione: Roupas novas (especifique os tipos: camisas, calças, etc.). Detalhe ao máximo para facilitar a compra.
> *   **Tarefas:**
>     *   **Consertar tomadas e a cortina rasgada da sala:** Divida em subtarefas como "Comprar materiais para tomadas", "Agendar eletricista (se necessário)", "Avaliar se a cortina pode ser consertada" ou "Comprar nova cortina". Defina prazos.
>     *   **Intercâmbio para a Inglaterra:** Crie subtarefas como "Pesquisar programas de intercâmbio", "Verificar requisitos", "Calcular custos", "Definir tipo de intercâmbio e instituições".
>     *   **Novo projeto para trabalhar com IA:** Subtarefas: "Definir área de IA", "Pesquisar ferramentas necessárias", "Criar plano de estudos/curso online".
>     *   **Voltar a ler:** Subtarefas: "Definir tempo diário para leitura", "Escolher o primeiro livro".
>     *   **Melhorar meus tempos para estudos:**
>         *   Desenho: "Criar cronograma de estudos de desenho", "Identificar áreas de atenção", "Encontrar materiais de apoio".
>         *   Automação de testes: "Criar plano de estudos para automação de testes", "Escolher ferramentas", "Praticar com projetos".
>     *   **Separar roupas para doação:** Agende um dia/hora para revisar o guarda-roupa e separar as peças.
>     *   **Arrumar a planta da sala:** Subtarefas: "Colocar tutor na planta", "Criar muda da planta".
> 
> **Google Agenda:**
> 
> *   **Evento de astronomia (dia 25 às 19h):** Crie o evento com lembretes (1 dia antes e 2 horas antes). Inclua detalhes como local e o que levar.
> *   **Ligar para o João (segunda-feira):** Agende um lembrete para segunda-feira de manhã.
> *   **Revisão Semanal:** Reserve um tempo fixo no final de cada semana para revisar suas tarefas e ajustar o plano.
> *   **Treino ciclismo:** Agende os horários de treino, se forem fixos, ou crie lembretes para os dias de treino.
> 
> **Google Keep:**
> 
> *   **Ideias e outros:** Use para armazenar informações gerais, links de programas de intercâmbio, ideias para o projeto de IA, anotações sobre os livros que quer ler, etc.
>     *   Crie notas separadas para cada projeto/área, como "Intercâmbio Inglaterra - Ideias", "Projeto IA - Ferramentas", "Leitura - Próximos Livros".
> 
> Ao organizar seus itens nessas ferramentas, você terá uma visão clara de suas tarefas, compromissos e ideias, facilitando o acompanhamento e a execução de cada um. Lembre-se de revisar suas tarefas e compromissos regularmente para manter tudo atualizado e garantir que você esteja no caminho certo para alcançar seus objetivos.
> 


-------------------------------------------------------------
